In [4]:
import os
import numpy as np
import pandas as pd
import sklearn
from sklearn import *
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as ms
ms.use('seaborn-muted')
from IPython.display import clear_output, display

# Create Training Dataset Through OSC

In [36]:
from pythonosc import dispatcher
from pythonosc import osc_server

data = pd.DataFrame(columns=['x', 'y', 'z'])
# To keep track for data collection
index = 0

x = 0
y = 0
z = 0

#creating a function that will handle and classify accelerometer data
def acceleration_vector(address, args):
    global data
    global index
    global x
    global y
    global z
    if address.find('accelerometer/x') != -1:
        x = args
    elif address.find('accelerometer/y') != -1:
        y = args
    elif address.find('accelerometer/z') != -1:
        z = args
        data = data.append({'x': x, 'y': y, 'z': z}, ignore_index=True)
        clear_output(wait=True)
        print(index)
        index = index + 1

#attaching the function to the dispatcher
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/accelerometer/*", acceleration_vector)

#starting the OSC server
server = osc_server.ThreadingOSCUDPServer(('193.157.241.238', 8000), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

KeyboardInterrupt: 

7978


In [37]:
server.server_close()

In [40]:
data.to_csv('out.csv')

# Train the MLP Regressor

In [48]:
dataset = pd.read_csv('./data/followUp7/data.csv')

rawdata = dataset[['x', 'y', 'z']].to_numpy()
inputs = np.empty((0,3))
target = np.empty((inputs.shape[0],2))

# Bounds were specified manually through looking at the CSV file
for i in range(0,rawdata.shape[0]):
    if (2 <= i <= 1025):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[1.,1.]]), axis=0) #looking at phone
    elif (1052 <= i <= 1990):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[1.,0.]]), axis=0) #face level to the left
    elif (2007 <= i <= 3010):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[0.,1.]]), axis=0) #face level upwards
    elif (3050 <= i <= 3937):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[0.,0.]]), axis=0) #face level downwards

# Backup data
# for i in range(0,rawdata.shape[0]):
#     if (4056 <= i <= 5021):
#         inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
#         target = np.append(target, np.array([[1.,1.]]), axis=0) #arm down
#     elif (5051 <= i <= 5950):
#         inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
#         target = np.append(target, np.array([[1.,0.]]), axis=0) #arm side face down
#     elif (6006 <= i <= 6998):
#         inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
#         target = np.append(target, np.array([[0.,1.]]), axis=0) #arm side face up
#     elif (7048 <= i <= 7978):
#         inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
#         target = np.append(target, np.array([[0.,0.]]), axis=0) #arm up

#creating train/test split
inputs_train, inputs_test, target_train, target_test = sklearn.model_selection.train_test_split(inputs, target, test_size=0.1)

#training the model
mlp = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(8,4), max_iter=20000, activation='logistic')
mlp.fit(inputs_train, target_train)
target_predict =  mlp.predict(inputs_test)


#print the number of misclassified samples, accuracy and complete report
print('r2 score on individual targets',sklearn.metrics.r2_score(target_test, target_predict, multioutput='raw_values'))

(3469, 2)
(386, 2)
r2 score on individual targets [0.99725622 0.97945509]


# Receive OSC data and predict using the MLP Regressor

In [51]:
from pythonosc import dispatcher
from pythonosc import osc_server

acc_vect = np.zeros((1,3))

#creating a function that will handle and push accelerometer through the regressor
def acceleration_vector(address, args):
    if address.find('accelerometer/x') != -1:
        acc_vect[0,0] = args
    elif address.find('accelerometer/y') != -1:
        acc_vect[0,1] = args
    elif address.find('accelerometer/z') != -1:
        acc_vect[0,2] = args
        clear_output(wait=True)
        pred = mlp.predict(acc_vect)
        print('Parameters   %.3f'%pred.flat[0], '  %.3f' %pred.flat[1])

#attaching the function to the dispatcher
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/accelerometer/*", acceleration_vector)

#starting the OSC server
server = osc_server.ThreadingOSCUDPServer(('193.157.241.238', 8000), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

OSError: [Errno 49] Can't assign requested address

In [52]:
server.server_close()